In [1]:
import os

In [2]:
%pwd

'c:\\Users\\91844\\Desktop\\text_summarizer\\Text_Summarization_Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\91844\\Desktop\\text_summarizer\\Text_Summarization_Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    tokenizer_path: Path
    metrics_file_name: Path

In [6]:
from Text_Summarization_Project.constants import *
from Text_Summarization_Project.utils.common  import read_yaml,create_directories


In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_path=config.model_path,
            tokenizer_path=config.tokenizer_path,
            metrics_file_name=config.metrics_file_name
        )

        return model_evaluation_config


In [8]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset ,load_from_disk
import torch
import pandas as pd
from tqdm import tqdm

In [9]:
import transformers
print(transformers.__version__)


4.55.2


In [10]:

import evaluate

metric = evaluate.load("rouge")   # example metric for summarization


In [13]:


device = "cuda" if torch.cuda.is_available() else "cpu"

class ModelEvaluation:
    def __init__(self, config):
        self.config = config
        self.device = device

    @staticmethod
    def generate_batch_sized_chunks(list_of_elements, batch_size):
        """Split the dataset into smaller batches that we can process simultaneously.
        Yield successive batch-sized chunks from list_of_elements.
        """
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i : i + batch_size]

    def calculate_metric_on_test_ds(self, dataset, metric, model, tokenizer,
                                    batch_size=16,
                                    column_text="dialogue", column_summary="summary"):
        article_batches = list(self.generate_batch_sized_chunks(dataset[column_text], batch_size))
        target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary], batch_size))

        for article_batch, target_batch in tqdm(
            zip(article_batches, target_batches), total=len(article_batches)
        ):
            inputs = tokenizer(
                article_batch,
                max_length=512,
                truncation=True,
                padding="max_length",
                return_tensors="pt"
            )

            summaries = model.generate(
                input_ids=inputs["input_ids"].to(self.device),
                attention_mask=inputs["attention_mask"].to(self.device),
                length_penalty=0.8,
                num_beams=8,
                max_length=128
            )

            decoded_summaries = [
                tokenizer.decode(s, skip_special_tokens=True, clean_up_tokenization_spaces=True)
                for s in summaries
            ]

            decoded_summaries = [d.replace("", " ") for d in decoded_summaries]

            metric.add_batch(predictions=decoded_summaries, references=target_batch)

        score = metric.compute()
        return score
    
    def evaluate(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)
        
        dataset_samsum_pt = load_from_disk(self.config.data_path)
        
        rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
        rouge_metric = evaluate.load("rouge")
        
        score = self.calculate_metric_on_test_ds(
            dataset_samsum_pt["test"], rouge_metric, model_pegasus, tokenizer, batch_size=2,column_text="dialogue", column_summary="summary"
        )
        
        rouge_dict = {rn: score[rn] for rn in rouge_names}
        
        df = pd.DataFrame(rouge_dict, index=["pegasus"])
        df.to_csv(self.config.metrics_file_name, index=False)
        
        

In [14]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.evaluate()
except Exception as e:
    raise e

[2025-08-22 17:22:59,611 : INFO : common : yaml file: config\config.yaml loaded successfully]
[2025-08-22 17:22:59,620 : INFO : common : yaml file: params.yaml loaded successfully]
[2025-08-22 17:22:59,623 : INFO : common : created directory at: artifacts]
[2025-08-22 17:22:59,627 : INFO : common : created directory at: artifacts/model_evaluation]


  0%|          | 2/410 [01:54<6:29:54, 57.34s/it]


KeyboardInterrupt: 